In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2024-07-21 20:30:57.663153: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-21 20:30:57.832845: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-21 20:30:57.893994: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-21 20:30:57.913870: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-21 20:30:58.043277: I tensorflow/core/platform/cpu_feature_guar

In [2]:
print(tf.__version__)

2.17.0


In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


I0000 00:00:1721574061.395601    3881 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721574061.504485    3881 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721574061.509423    3881 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1721574061.514681    3881 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [4]:
titanic = pd.read_csv(r"/home/subham/Deep learning/AI/dnn-python-from-scratch/final_dataset_ready.csv")
dataset = titanic.copy()
X = np.asarray(dataset.drop('Survived',axis=1))
y = np.asarray(dataset['Survived'])

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,shuffle=True,random_state=0)

In [6]:
sc = StandardScaler()
X_trainNormalized = sc.fit_transform(X_train)
X_testNormalized = sc.transform(X_test)

In [7]:
numDims = X_trainNormalized.shape[1]
dnnModel = tf.keras.Sequential([
    layers.Dense(16,activation='relu',input_shape=(numDims,)),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(16,activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(126,activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(160,activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(10,activation='relu'),
    layers.Dropout(0.2),
    layers.BatchNormalization(),
    layers.Dense(2)
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
dnnModel.compile(optimizer=optimizer,loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['Accuracy'])

/home/subham/.conda/envs/train_dl/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
dnnModel.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 126)            │         2,142 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 126)            │           504 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 126)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 160)            │        20,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 160)            │           640 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 160)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │         1,610 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 10)             │            40 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 2)              │            22 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,838 (100.93 KB)

 Trainable params: 25,182 (98.37 KB)

 Non-trainable params: 656 (2.56 KB)

In [9]:
dnnModel.fit(X_trainNormalized,y_train,epochs=100)

Epoch 1/100


I0000 00:00:1721574652.510769    3939 service.cc:146] XLA service 0x7fb55c002450 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721574652.510795    3939 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3050, Compute Capability 8.6
2024-07-21 20:40:52.562897: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-21 20:40:52.793471: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902


 1/23 ━━━━━━━━━━━━━━━━━━━━ 1:11 3s/step - Accuracy: 0.5312 - loss: 1.2721

I0000 00:00:1721574654.511218    3939 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 100ms/step - Accuracy: 0.5476 - loss: 1.1021
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 870us/step - Accuracy: 0.5756 - loss: 0.8931
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 805us/step - Accuracy: 0.6053 - loss: 0.8170
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step - Accuracy: 0.6668 - loss: 0.7280
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step - Accuracy: 0.6779 - loss: 0.7219
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step - Accuracy: 0.6783 - loss: 0.7003
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 981us/step - Accuracy: 0.6908 - loss: 0.6627
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 878us/step - Accuracy: 0.6812 - loss: 0.6956
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 834us/step - Accuracy: 0.7082 - loss: 0.6161
Epoch 10/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 869us/step - Accuracy: 0.7170 - loss: 0.6354
Epoch 11/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 888us/step - Accuracy: 0.7119 - loss: 0.6084
Epoch 12/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 879us/step

In [10]:
test_loss , test_acc = dnnModel.evaluate(X_testNormalized,y_test,verbose=2)
print('\nTest Accuracy :',test_acc)

6/6 - 2s - 303ms/step - Accuracy: 0.8101 - loss: 0.4190

Test Accuracy : 0.8100558519363403


In [11]:
y_pred = np.argmax(dnnModel.predict(X_test), axis=-1)

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step


In [12]:
idx = np.random.randint(1,len(y_test))
print('Index : ',idx,'Actual value : ',y_test[idx],'Predicted : ',y_pred[idx])

Index :  82 Actual value :  0 Predicted :  0


In [13]:
dnnModel.predict(X_test).shape

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step


(179, 2)